# Summary analysis: Intermittent motion - frequency tuning
## effect of social isolation


In [ ]:
%config InteractiveShellApp.pylab_import_all = False
%matplotlib inline
%pylab inline
%reload_ext autoreload
%autoreload 2

import sys
import os
import fnmatch

import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame, Series
import seaborn as sns
import glob
from datetime import datetime

if 'startDirMaster' not in locals():
    startDirMaster=os.getcwd()

propsFn=startDirMaster+'\\props.csv'
props=pd.read_csv(propsFn, header=None, index_col=0, squeeze=True,delim_whitespace=True).to_dict()

base=props['BaseDir']
expFile=props['allExpFn']

RawDataDir = os.path.join(base,props['RawDataDir'])+'\\'
ProcessingDir = os.path.join(base,props['ProcessingDir'])+'\\Fig2E_isolation\\'
outputDir = os.path.join(base,props['outputDir'])+'\\'

if not os.path.isdir(ProcessingDir):
    os.makedirs(ProcessingDir)
if not os.path.isdir(outputDir):
    os.makedirs(outputDir)

os.chdir('..\\')
import functions.matrixUtilities_joh as mu
import matplotlib.pyplot as plt
import models.experiment as xp
import models.experiment_set as es
import functions.paperFigureProps as pfp
pfp.paper()
inToCm=2.54


In [ ]:
info=pd.read_csv(expFile, sep=',')#pd.read_csv(expFile,quotechar='"', sep=',', converters={'bdGroup':ast.literal_eval})
info=info[info.stimulusProtocol=='3dIso']
info

In [ ]:
condition=np.array([1,0,1,0,1,
                   0,1,0,1,0,
                   1,0,1,0,1,
                   1,0,1,0,1,
                   0,1,0,1,0,
                   1,0,1,0,1])

In [ ]:
# collect meta information and save to new csv file for batch processing

aviPath=[]
posPath=[]
PLPath=[]
expTime = []
    
for index,row in info.iterrows():
    startDir=RawDataDir+row.path+'\\'
    #startDir='D:\\data\\b\\2017\\'+row.path+'\\'
    #if not os.path.isdir(startDir):
    #    startDir='E:\\b\\2017\\'+row.path+'\\'
        
    posPath.append(glob.glob(startDir+'PositionTxt*.txt')[0])
    PLPath.append(glob.glob(startDir+'PL*.txt')[0])
    
    head, tail = os.path.split(posPath[-1])
    currTime=datetime.strptime(tail[-23:-4], '%Y-%m-%dT%H_%M_%S')
    expTime.append(currTime)
    
info['txtPath']=posPath
info['pairList']=PLPath

info['epiDur'] = 5      # duration of individual episodes (default: 5 minutes)
info['episodes'] = -1   # number of episodes to process: -1 to load all episodes (default: -1)
info['inDish'] = 10#np.arange(len(posPath))*120     # time in dish before experiments started (default: 10)
info['arenaDiameter_mm'] = 100 # arena diameter (default: 100 mm)
info['minShift'] = 60 # minimum number of seconds to shift for control IAD
info['episodePLcode'] = 0 # flag if first two characters of episode name encode animal pair matrix (default: 0)
info['recomputeAnimalSize'] = 1 # flag to compute animals size from avi file (takes time, default: 1)
info['SaveNeighborhoodMaps'] = 0 # flag to save neighborhood maps for subsequent analysis (takes time, default: 1)
info['computeLeadership'] = 0 # flag to compute leadership index (takes time, default: 1)
info['ComputeBouts'] = 1 # flag to compute swim bout frequency (takes time, default: 1)
info['set'] = np.arange(len(posPath))   # experiment set: can label groups of experiments (default: 0)
info['ProcessingDir']=ProcessingDir
info['outputDir']=outputDir

info['expTime']=expTime

csvFile=os.path.join(ProcessingDir,'Fig2_E_isolation.csv')
info.to_csv(csvFile,encoding='utf-8')
info

In [ ]:
def readExperiment(keepData=False):
    tmp=es.experiment_set(csvFile=csvFile)
    if keepData:
        return tmp
    else:
        return 1

expSet=readExperiment(keepData=True)

In [ ]:
csvPath = []
for f in [mu.splitall(x)[-1][:-4] for x in info.txtPath]:
    csvPath.append(glob.glob(ProcessingDir+f+'*siSummary*.csv')[0])

df=pd.DataFrame()
i=0
for fn in csvPath:
    print(fn)
    tmp=pd.read_csv(fn,index_col=0,sep=',')
    tmp.animalSet=i
    tmp.animalIndex=tmp.animalIndex+((i)*15)
    df=pd.concat([df,tmp])
    i+=1
df['episode']=[x.strip().replace('_','') for x in df['episode']]

print('df shape',df.shape)

In [ ]:
d=df.time
r=datetime(2017,1,1)
t2=[pd.to_datetime(x).replace(day=1,month=1)for x in df.time]
t3=[(x-r)/pd.Timedelta('1 hour') for x in t2]
df['t2']=t2
df['t3']=t3
df.head()

## Habituation or Fatigue within 20 hours?

Plot shoaling index during closed loop skype episodes over time.

In [ ]:
sns.tsplot(data=df, time="inDishTime",value="si",unit="animalIndex",condition="episode",estimator=np.nanmean,interpolate=False,err_style="ci_bars");
plt.xlim([0,8*60])

In [ ]:
df['condition']=condition[df.animalIndex]
sns.tsplot(data=df[df.episode=='01real'], time="inDishTime",value="si",unit="animalIndex",condition="condition");

# Mean response over all stimuli per animal

In [ ]:
sns.set_palette('viridis',30)
co=sns.color_palette("viridis", 30)
idx=(df['inDishTime']<400) & (df['inDishTime']>45)
dfDR=df[idx]
dfEpiAn=dfDR.groupby(['episode','animalIndex'],sort=True).mean().reset_index()
sns.stripplot(data=dfEpiAn,x='episode',y='si',zorder=-1,hue='animalIndex')
sns.pointplot(data=dfEpiAn,x='episode',y='si',hue='animalIndex',zorder=100,scale=0.2,palette=['gray'])
sns.pointplot(data=dfEpiAn,x='episode',y='si',join=False,zorder=100)
ax=plt.gca()
ax.legend_.remove()

In [ ]:
sns.set_palette('viridis',2)
co=sns.color_palette("viridis", 2)
fig, axes = plt.subplots(figsize=(5, 5))
sns.pointplot(data=dfEpiAn,x='episode',y='si',hue='condition',zorder=100,scale=1)
sns.despine()

axes.set_ylabel('attraction index')
axes.axhline(0,ls=':',color='k')
axes.set_title('Frequency tuning per age group');
plt.legend(title='age')

In [ ]:
xax=np.array([-.5, 1/30., 1./(30./5), 1./(30./10), 1./(30./20), 1./(30./30), 1./(30./40), 1./(30./50)])

g_epiAg=dfEpiAn.groupby(['episode','condition'],sort=True)[['si']]
var_group=g_epiAg.std().unstack().values.T
si_group=g_epiAg.mean().unstack().reset_index()

si_group['xax']=xax
fig, axes = plt.subplots(figsize=(5, 5))

axes=si_group.plot(x='xax',
                   y='si',
                   kind='line',
                   marker='o',
                   linestyle=':',
                   ax=axes,
                   legend=True)

axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('group frequency tuning');

In [ ]:
col=['gray','k']

fig, axes = plt.subplots(figsize=(4.5/inToCm,4.5/inToCm))

# plot dots for age groups
si_group.plot(kind='line',
            marker='o',
            ls='',
            x='xax',
            y='si',
            ax=axes,
            ms=5,
            markeredgecolor='k',
            markeredgewidth=1,
            legend=True,
             color = col)

#connect frequency tuning with a line
si_group[si_group.xax>0].plot(kind='line',
            x='xax',
            y='si',
            ax=axes,
            ms=0,
            legend=False,
            label='',
                             color=col)

# draw error bars (+/- 1SD)
ii=0
for i in [0,1]:
    y=si_group.si.iloc[:,i].values
    x=xax+(ii-.5)*0.05
    
    e=var_group[i]
    c=col[i]
    axes.errorbar(x,y,e,fmt='none',color=c,alpha=0.5)
    ii+=1
    
axes.set_xticks(xax);
xls='real %.2f %.1f %.1f %.1f %.1f %.1f %.1f' % tuple(xax[1:])
xls=[x.lstrip('0') for x in xls.split()]
xls[2]=''
xls[5]='1'
axes.set_xticklabels(xls,fontsize=8);
axes.set_yticks([0,.2,.4]);
axes.set_ylim([-.15,.5]);
axes.set_xlim([-0.7,2])
axes.set_xlabel('Bout interval (sec)')
axes.set_title('Isolated animals')
axes.set_ylabel('Attraction')
fig.subplots_adjust(top=0.75)
sns.despine()
#plt.legend(loc='upper right')

# pretty Legend
handles, labels = axes.get_legend_handles_labels()
labels=np.array(labels)
handles=np.array(handles)
li=np.array([0,1])
L = plt.legend(handles[li],
               labels[li], 
               bbox_to_anchor=(.6, 1.05), 
               loc=2, 
               borderaxespad=0.,
               title='Rearing')
L.get_texts()[0].set_text('Group')
L.get_texts()[1].set_text('Isolated')
axes.axhline(0,ls=':',color='gray')

figPath=outputDir+'\\2Ea_BoutFreq_isolated.svg'
plt.savefig(figPath)

In [ ]:
sns.set_palette('viridis',30)
co=sns.color_palette("viridis", 30)

g_epiAn=dfEpiAn.groupby(['episode','animalIndex'],sort=True)[['si']]
si_an=g_epiAn.mean().unstack().reset_index()
si_an['xax']=xax

fig, axes = plt.subplots(figsize=(3, 3))

axes=si_an.plot(x='xax',
                   y='si',
                   kind='line',
                   marker='.',
                   yerr=0,
                   linestyle=':',
                   ax=axes,
                   legend=True)

axes.set_ylabel('attraction index')
plt.xlim([0,xax.max()+xax.max()*0.1])
axes.axhline(0,ls=':',color='k')
axes.set_title('group frequency tuning');

axes.legend_.remove()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=15, sharex=True, sharey=True,figsize=(30/inToCm,10/inToCm))
ax=ax.ravel()
col1=['gray','k']

maxPosAllRawMax=[]
maxPosAllRawMaxTr=[] 
ageAll=[]
x=xax[1:]

for i in range(si_an.si.shape[1]):
    y=si_an.si.values[1:,i]
    z=np.polyfit(x,y,2)
    p = np.poly1d(z)
    xp = np.linspace(0, 1.7, 1000)
    ax[i].plot(x, y, '.', xp, p(xp), '-')
    ax[i].set_ylim([-.20,.6])
    ax[i].axis('off')
    ax[i].set_title('Animal '+str(i))
    putativeMax=x[np.argmax(y)]
    maxPosAllRawMax.append(putativeMax) # all animals
    if (y.max()>0.05): #only animals above threshold 
        maxPosAllRawMaxTr.append(putativeMax)
        interpolated=np.argmax(p(xp))/(1000/1.7)
        ax[i].axvline(interpolated)
        ax[i].axvline(maxPosAllRawMaxTr[-1],color='r')
    else:
        maxPosAllRawMaxTr.append(np.nan)
    ageAll.append(df[df.animalIndex==si_an.si.columns[i]].condition.values[0])

mpa=pd.DataFrame({'condition':ageAll,'mp':maxPosAllRawMaxTr}) #mpa: max per animal, only animals above threshold
maxPosIndMn=mpa.groupby(['condition']).mean().mp
maxPosIndSTD=mpa.groupby(['condition']).std().mp
print([maxPosIndMn,maxPosIndSTD])

mpaAll=pd.DataFrame({'condition':ageAll,'mp':maxPosAllRawMax}) #mpa: max per animal, all animals
maxPosIndMnAll=mpaAll.groupby(['condition']).mean().mp
maxPosIndSTDAll=mpaAll.groupby(['condition']).std().mp
print([maxPosIndMnAll,maxPosIndSTDAll])

In [ ]:
#visualize maxima per over age for individual animals. Note discrete max levels

sns.jointplot(mpa.condition,mpa.mp,alpha=0.2)

In [ ]:

mpa.head()

In [ ]:
fig, ax = plt.subplots(sharex=True, sharey=True,figsize=(2/inToCm,4.5/inToCm))
col1=['gray','k']


sns.swarmplot(data=mpa,x='condition',y='mp',ax=ax,palette=col1,zorder=0,size=2.3)
sns.pointplot(data=mpa,x='condition',y='mp',ax=ax,palette=['k'],zorder=100,markers=['_'],scale=3,errwidth=1,capsize=.3)
sns.pointplot(data=mpa,x='condition',y='mp',ax=ax,palette=['r'],zorder=100,markers=['_'],scale=3,ci=None)
plt.ylabel('Optimal interval (sec)')
plt.xlabel('')
plt.yticks(np.arange(0,1.8,.4))
ax.set_ylim([0,1.6])

ax.set_xticklabels(['Group','Isolated'])

for tick in ax.xaxis.get_major_ticks():
    #tick.label.set_fontsize(12) 
    tick.label.set_rotation(45)
    
#plt.xticks([])
sns.despine()

x1, x2 = 0, 1   # columns
l=0.05
y, h, col = mpa.mp.max() + l, l, 'k'
plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
plt.text((x1+x2)*.5, y+h, "ns", ha='center', va='bottom', color=col,size=10)


figPath=outputDir+'\\2Eb_BoutFreq_isolatedBar.svg'
plt.savefig(figPath)

In [ ]:
maxPosIndMn=mpa.groupby(['condition']).mean().mp
maxPosIndSTD=mpa.groupby(['condition']).std().mp
maxPosIndMn

In [ ]:
forPrism=mpa.copy().sort_values(by=['condition'])
forPrism['an']=np.concatenate([np.arange(14),np.arange(16)])
forPrism
forPrism=forPrism.pivot_table(columns='condition',values='mp',index='an').reset_index()
forPrism.to_csv(outputDir+'Fig2E.csv')
forPrism

# Prism statistical analysis

Table Analyzed	Data 1
	
Column B	Data Set-B
vs.	vs.
Column A	Data Set-A
	
Mann Whitney test	
  P value	0.5463
  Exact or approximate P value?	Exact
  P value summary	ns
  Significantly different (P < 0.05)?	No
  One- or two-tailed P value?	Two-tailed
  Sum of  ranks in column A,B	229.5 , 235.5
  Mann-Whitney U	99.5
	
Difference between medians	
  Median of column A	0.6667, n=14
  Median of column B	0.6667, n=16
  Difference: Actual	0
  Difference: Hodges-Lehmann	0

from shutil import copy2

def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts



for i,row in info.iterrows():
    fn=row.txtPath
    head, tail = os.path.split(fn)

    copyList=[]
    copyList.append(glob.glob(head+'\\ROI*.csv')[0])
    copyList.append(glob.glob(head+'\\PositionTxt*.txt')[0])
    copyList.append(glob.glob(head+'\\PL*.txt')[0])
    copyList.append(glob.glob(head+'\\*anSize.csv')[0])
    
    for f in copyList:
        print(f)
        if f[0]=='E':
            keepSlash=3
        else:
            keepSlash=4
        toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" + os.path.join(*splitall(f)[keepSlash:-1])+"\\"
        #toDirectory = "e:\\b\\LarschAndBaier2018\\RawData\\" 
        if not os.path.isdir(toDirectory):
            os.makedirs(toDirectory)
        
        copy2(f, toDirectory)
